In [1]:
#import libraries needed
#!conda install -c conda-forge beautifulsoup4 --yes #uncomment if install needed
from bs4 import BeautifulSoup #library for web scraping
import requests

import numpy as np #for handling data in vectorized manner

import pandas as pd #for data analysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes #uncomment if install needed
import folium # map rendering library

print('Libraries imported.')


Libraries imported.


# Getting the webpage, parsing it using BeautifulSoup and creating the DataFrame

In [2]:
#use requests to get the link, and parse using BeautifulSoup
#link = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(url, 'html.parser')
#creating the new dataframe

po_df = pd.DataFrame()
i = 0
end = False 
#get the rows of table
for tr in soup.find_all('tr'):# find the <tr> tag and loop through to get rows
    j = 0 
    for tds in tr.find_all('td'):#loop <td> tag to get columns of row
        if (len(tds.text) > 1)& (not end): #make sure to get row we want
            po_df.loc[i,j]=tds.text 
            j = j+1
        else:
            end = True
    i =i+1


po_df.columns =['PostalCode', 'Borough', 'Neighbourhood']

print('shape of data frame is:', po_df.shape)

print(po_df.head())
            
    




shape of data frame is: (288, 3)
  PostalCode           Borough       Neighbourhood
1        M1A      Not assigned      Not assigned\n
2        M2A      Not assigned      Not assigned\n
3        M3A        North York         Parkwoods\n
4        M4A        North York  Victoria Village\n
5        M5A  Downtown Toronto      Harbourfront\n


# Cleaning Data

In [3]:
# remove \n, format data base look
po_df['Neighbourhood']=po_df['Neighbourhood'].str.replace('\n', '')
po_df=po_df.reset_index(drop=True)
po_df.head(12)

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


In [4]:
#remove cells with borough not assigned 
po_df = po_df.drop(po_df.index[po_df['Borough']=='Not assigned'])

po_df.head(11)

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [5]:
#Merge neighbourhoods with the same postcode using .groupby function and a lambda function
post_df = po_df.groupby('PostalCode', sort=False, as_index=False).agg(lambda x:', '.join(set(x)))
post_df.head(12)

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Not assigned
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [6]:
#assign borough to neighbourhoods that are not assigned
for i in range(post_df.shape[0]):
    if post_df.loc[i,'Neighbourhood'] =='Not assigned':
        post_df.loc[i,'Neighbourhood'] = post_df.loc[i,'Borough']
post_df.head(11)

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [7]:
post_df.shape

(103, 3)

# Add Latituide and Longitdue data to post_df Data fram

In [8]:
#Attempted to use geocoder, with the following code, but it did not work:
"""
import geocoders
# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]
"""
#use csv file instead:

co_df = pd.read_csv('https://cocl.us/Geospatial_data')
print(co_df.head())
print('shape:',co_df.shape) #check the shape to make sure it matches post_df

  Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476
shape: (103, 3)


# Match the headings of both data frames and Merge them

In [9]:
co_df.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
co_df.head()#check if changes applied 

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
#sort both dataframes by PostCode to make sure the line up
co_df.sort_values(by = ['PostalCode'], inplace=True)
post_df.sort_values(by = ['PostalCode'], inplace=True)

In [11]:
#merge the data frames
Toronto_df = post_df.merge(co_df, left_on='PostalCode', right_on='PostalCode', )
Toronto_df.head(12)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Morningside, West Hill, Guildwood",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Oakridge, Golden Mile, Clairlea",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Scarborough Village West, Cliffcrest",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [12]:
Toronto_df.shape

(103, 5)

# Exploring the Data

In [13]:
#get the longitude and latitude data for Toronto
address = 'Toronto, Canada'
geolocater = Nominatim(user_agent = 'ny_explorer')
location = geolocater.geocode(address)
latitude = location.latitude
longitude = location.longitude 
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))


The geograpical coordinates of Toronto are 43.653963, -79.387207.


#### Create a map of Toronto with neighborhoods superimposed on top

In [14]:
#create the map
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

#Add markers
for lat, lng, borough, neighborhood, in zip(Toronto_df['Latitude'], Toronto_df['Longitude'],
                                           Toronto_df['Borough'], Toronto_df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#31866cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)
    
map_toronto


In [15]:
#find all the Boroughs in Toronto
print(Toronto_df['Borough'].unique())

['Scarborough' 'North York' 'East York' 'East Toronto' 'Central Toronto'
 'Downtown Toronto' 'York' 'West Toronto' "Queen's Park" 'Mississauga'
 'Etobicoke']


#### Let's create a data frame for North York and visiualize the Neighbourhoods in it

In [16]:
#create a new dataframe with North York values
N_df = Toronto_df[Toronto_df['Borough'] == 'North York'].reset_index(drop=True)
N_df.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M2H,North York,Hillcrest Village,43.803762,-79.363452
1,M2J,North York,"Fairview, Oriole, Henry Farm",43.778517,-79.346556
2,M2K,North York,Bayview Village,43.786947,-79.385975
3,M2L,North York,"York Mills, Silver Hills",43.757490,-79.374714
4,M2M,North York,"Willowdale, Newtonbrook",43.789053,-79.408493


In [17]:
#get the address
address = 'North York, Toronto'
geolocator = Nominatim(user_agent='ny_explorer')
location=geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map of north york using latitude and longitude values
map_n = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, label in zip(N_df['Latitude'], N_df['Longitude'], N_df['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_n)  
    
map_n

Next we are going to utilize the Foursquare API to explore the neighbourhoods and segment them 

#### Define Foursquare Credentials and Version

In [75]:
#REMOVED FOR SECURITY REASONS
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


#### Explore  a neighbourhood in the dataframe

Get the neighbourhood's name.

In [19]:
#get the neighbourhood's name
N_df.loc[20, 'Neighbourhood']

'Glencairn'

Get the neighborhood's latitude and longitude values.

In [20]:
neighborhood_latitude = N_df.loc[20, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = N_df.loc[20, 'Longitude'] # neighborhood longitude value

neighborhood_name = N_df.loc[20, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Glencairn are 43.709577, -79.44507259999999.


Now, let's get the top 100 venues that are in Glencairn within a radius of 1 km.


In [21]:
LIMIT = 100
radius = 1000
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit= {}'.format(CLIENT_ID,
    CLIENT_SECRET,
    VERSION,                                                                                                                         
    neighborhood_latitude,
    neighborhood_longitude,
    radius,
    LIMIT)
    

url


'https://api.foursquare.com/v2/venues/explore?&client_id=M3FLJYCCTBLUK5BIFFJY1WKSOF0UFXB2QBIHT51WIDAJGRPP&client_secret=24ASR43ML2QXF5I01MRGRD0D4MZP04WHLBWZ13IEMTLANQVV&v=20180605&ll=43.709577,-79.44507259999999&radius=1000&limit= 100'

In [22]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d1aa17d531593002ca0c781'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Briar Hill - Belgravia',
  'headerFullLocation': 'Briar Hill - Belgravia, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 36,
  'suggestedBounds': {'ne': {'lat': 43.718577009000015,
    'lng': -79.43264514701556},
   'sw': {'lat': 43.70057699099999, 'lng': -79.45750005298441}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4fca732be4b07c8e3e198c27',
       'name': 'Miyako Sushi Restaurant',
       'location': {'address': '572 Marlee Ave',
        'crossStreet': 'at Glencairn Ave',
        'lat': 43.709111,
        'lng': -79.44393,
        'l

All the information is in the items key but, before we proceed, let's define a function get_category_type to retrieve the category of venues.

In [23]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we can clean the json file and structure it into a pandas dataframe

In [24]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Miyako Sushi Restaurant,Japanese Restaurant,43.709111,-79.443930
1,Viewmount Park,Playground,43.707214,-79.438229
2,Columbus Colombian Bakery (Panaderia Colombiana),Latin American Restaurant,43.709173,-79.453331
3,Starbucks,Coffee Shop,43.715713,-79.443717
4,Dairy Queen,Fast Food Restaurant,43.710068,-79.453441


In [25]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))


36 venues were returned by Foursquare.


## Explore Neighborhoods in North York


Let's create a function to repeat the process above with each neighbourhood in North york

In [26]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)



Now we write the code to run the above function on each neighborhood and create a new dataframe called north_venues

In [27]:
north_venues = getNearbyVenues(names=N_df['Neighbourhood'],
                                   latitudes=N_df['Latitude'],
                                   longitudes=N_df['Longitude']
                                  )


Hillcrest Village
Fairview, Oriole, Henry Farm
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview East, CFB Toronto
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Lawrence Manor East, Bedford Park
Lawrence Heights, Lawrence Manor
Glencairn
Downsview, North Park, Upwood Park
Humber Summit
Humberlea, Emery


In [28]:
print(north_venues.shape)
north_venues.head()

(251, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Hillcrest Village,43.803762,-79.363452,Eagle's Nest Golf Club,43.805455,-79.364186,Golf Course
1,Hillcrest Village,43.803762,-79.363452,AY Jackson Pool,43.804515,-79.366138,Pool
2,Hillcrest Village,43.803762,-79.363452,Villa Madina,43.801685,-79.363938,Mediterranean Restaurant
3,Hillcrest Village,43.803762,-79.363452,Duncan Creek Park,43.805539,-79.360695,Dog Run
4,"Fairview, Oriole, Henry Farm",43.778517,-79.346556,The LEGO Store,43.778207,-79.343483,Toy / Game Store


let's check how many venues were returned for each neighbourhood

In [29]:
north_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Wilson Heights, Downsview North",18,18,18,18,18,18
Bayview Village,4,4,4,4,4,4
Don Mills North,5,5,5,5,5,5
Downsview Central,4,4,4,4,4,4
"Downsview East, CFB Toronto",3,3,3,3,3,3
Downsview Northwest,5,5,5,5,5,5
Downsview West,3,3,3,3,3,3
"Downsview, North Park, Upwood Park",5,5,5,5,5,5
"Fairview, Oriole, Henry Farm",66,66,66,66,66,66


Check the amount of unique catagories 

In [30]:
print('There are {} uniques categories.'.format(len(north_venues['Venue Category'].unique())))

There are 111 uniques categories.


## Analyze Each Neighborhood

In [31]:
# one hot encoding
n_onehot = pd.get_dummies(north_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
n_onehot['Neighborhood'] = north_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [n_onehot.columns[-1]] + list(n_onehot.columns[:-1])
n_onehot = n_onehot[fixed_columns]

n_onehot.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bike Shop,Boutique,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Cafeteria,Café,Candy Store,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Empanada Restaurant,Event Space,Falafel Restaurant,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,Fraternity House,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,General Entertainment,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Kids Store,Korean Restaurant,Liquor Store,Lounge,Luggage Store,Massage Studio,Mediterranean Restaurant,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Other Repair Shop,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shoe Store,Shopping Mall,Smoke Shop,Smoothie Shop,Spa,Sporting Goods Shop,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store
0,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Fairview, Oriole, Henry Farm",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [32]:
n_onehot.shape

(251, 112)

Next, let's group rows by neighbourhood and by taking the mean of the frequency of occurrence of each category

In [33]:
n_grouped = n_onehot.groupby('Neighborhood').mean().reset_index()
n_grouped

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bike Shop,Boutique,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Cafeteria,Café,Candy Store,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Empanada Restaurant,Event Space,Falafel Restaurant,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,Fraternity House,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,General Entertainment,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Kids Store,Korean Restaurant,Liquor Store,Lounge,Luggage Store,Massage Studio,Mediterranean Restaurant,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Other Repair Shop,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shoe Store,Shopping Mall,Smoke Shop,Smoothie Shop,Spa,Sporting Goods Shop,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store
0,"Bathurst Manor, Wilson Heights, Downsview North",0.0000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.055556,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.00,0.0,0.0,0.000000,0.00,0.055556,0.000000,0.000000,0.000000,0.055556,0.000000,0.00,0.000000,0.000000,0.0000,0.000000,0.055556,0.000000,0.000000,0.00,0.0000,0.055556,0.055556,0.0000,0.000000,0.00,0.00,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.055556,0.000000,0.00,0.00,0.00,0.000000,0.055556,0.000000,0.055556,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000,0.055556,0.000000,0.00,0.000000,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000
1,Bayview Village,0.0000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.250000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.250000,0.000000,0.000000,0.250000,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.00,0.0000,0.000000,0.000000,0.0000,0.000000,0.00,0.00,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.250000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Don Mills North,0.0000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.200000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.200000,0.000000,0.200000,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.00,0.0000,0.000000,0.000000,0.0000,0.000000,0.00,0.00,0.000000,0.000000,0.2,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.200000,0.000000,0

In [34]:
n_grouped.shape

(23, 112)

Let's print each neighborhood along with the top 5 most common venues

In [35]:
num_top_venues = 5

for hood in n_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = n_grouped[n_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor, Wilson Heights, Downsview North----
                 venue  freq
0          Coffee Shop  0.11
1                Diner  0.06
2        Shopping Mall  0.06
3           Restaurant  0.06
4  Fried Chicken Joint  0.06


----Bayview Village----
                 venue  freq
0   Chinese Restaurant  0.25
1                 Bank  0.25
2                 Café  0.25
3  Japanese Restaurant  0.25
4    Accessories Store  0.00


----Don Mills North----
                  venue  freq
0  Gym / Fitness Center   0.2
1  Caribbean Restaurant   0.2
2                  Café   0.2
3        Baseball Field   0.2
4   Japanese Restaurant   0.2


----Downsview Central----
               venue  freq
0  Korean Restaurant  0.25
1       Home Service  0.25
2     Baseball Field  0.25
3         Food Truck  0.25
4             Lounge  0.00


----Downsview East, CFB Toronto----
               venue  freq
0            Airport  0.33
1               Park  0.33
2  Other Repair Shop  0.33
3                Gym  0.00
4

#### Let's put that into a pandas dataframe

First, let's write a function to sort the venues in descending order.

In [36]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.


In [37]:

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = n_grouped['Neighborhood']

for ind in np.arange(n_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(n_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Supermarket,Middle Eastern Restaurant,Frozen Yogurt Shop,Pharmacy,Pizza Place,Deli / Bodega,Restaurant,Bridal Shop,Sandwich Place
1,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Electronics Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store
2,Don Mills North,Gym / Fitness Center,Caribbean Restaurant,Café,Baseball Field,Japanese Restaurant,Women's Store,Electronics Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega
3,Downsview Central,Korean Restaurant,Home Service,Food Truck,Baseball Field,Women's Store,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store
4,"Downsview East, CFB Toronto",Park,Airport,Other Repair Shop,Dog Run,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store


## Cluster Neighbourhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [57]:
#set number of clusters
kclusters = 5

n_clustered = n_grouped.drop('Neighborhood', 1)

#run k-means clustering
kmeans = KMeans( n_clusters=kclusters, init='k-means++',random_state=0, n_init=10).fit(n_clustered)

#check cluster labels
kmeans.labels_


array([4, 0, 0, 0, 3, 0, 4, 3, 4, 4, 3, 4, 4, 2, 4, 4, 4, 3, 4, 4, 4, 3,
       1])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [60]:
# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_) 
n_merged = N_df
#merge
n_merged = n_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood', )
n_merged.reset_index
n_merged.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M2H,North York,Hillcrest Village,43.803762,-79.363452,4.0,Dog Run,Mediterranean Restaurant,Pool,Golf Course,Fried Chicken Joint,Dim Sum Restaurant,Comfort Food Restaurant,Construction & Landscaping,Furniture / Home Store,Convenience Store
1,M2J,North York,"Fairview, Oriole, Henry Farm",43.778517,-79.346556,4.0,Clothing Store,Fast Food Restaurant,Coffee Shop,Restaurant,Kids Store,Toy / Game Store,Tea Room,Metro Station,Bakery,Japanese Restaurant
2,M2K,North York,Bayview Village,43.786947,-79.385975,0.0,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Electronics Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store
3,M2L,North York,"York Mills, Silver Hills",43.757490,-79.374714,1.0,Cafeteria,Women's Store,Coffee Shop,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop
4,M2M,North York,"Willowdale, Newtonbrook",43.789053,-79.408493,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Let's visualize the Clusters 

In [65]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(n_merged['Latitude'], n_merged['Longitude'], n_merged['Neighbourhood'], n_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    print(cluster)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow,
        fill=True,
        fill_color=rainbow,
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

4.0
4.0
0.0
1.0
nan
4.0
3.0
4.0
3.0
0.0
4.0
4.0
4.0
3.0
4.0
0.0
0.0
4.0
4.0
4.0
3.0
3.0
4.0
2.0


## Examine Clusters

##### Cluster 1

In [67]:
n_merged.loc[n_merged['Cluster Labels'] == 0, n_merged.columns[[1] + list(range(5, n_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,North York,0.0,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Electronics Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store
9,North York,0.0,Gym / Fitness Center,Caribbean Restaurant,Café,Baseball Field,Japanese Restaurant,Women's Store,Electronics Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega
15,North York,0.0,Korean Restaurant,Home Service,Food Truck,Baseball Field,Women's Store,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store
16,North York,0.0,Grocery Store,Gym / Fitness Center,Athletics & Sports,Discount Store,Liquor Store,Electronics Store,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega


##### Cluster 2

In [70]:
n_merged.loc[n_merged['Cluster Labels'] == 1, n_merged.columns[[1] + list(range(5, n_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,North York,1.0,Cafeteria,Women's Store,Coffee Shop,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop


##### Cluster 3

In [71]:
n_merged.loc[n_merged['Cluster Labels'] == 2, n_merged.columns[[1] + list(range(5, n_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,North York,2.0,Baseball Field,Women's Store,Electronics Store,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop


##### Cluster 4

In [73]:
n_merged.loc[n_merged['Cluster Labels'] == 3, n_merged.columns[[1] + list(range(5, n_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,North York,3.0,Park,Convenience Store,Bank,Bar,Electronics Store,Construction & Landscaping,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store
8,North York,3.0,Park,Food & Drink Shop,Fast Food Restaurant,Women's Store,Dog Run,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega
13,North York,3.0,Park,Airport,Other Repair Shop,Dog Run,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store
20,North York,3.0,Pizza Place,Pub,Japanese Restaurant,Park,Women's Store,Diner,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop
21,North York,3.0,Park,Construction & Landscaping,Deli / Bodega,Bakery,Basketball Court,Electronics Store,Convenience Store,Cosmetics Shop,Cupcake Shop,Department Store


##### Cluster 5

In [74]:
n_merged.loc[n_merged['Cluster Labels'] == 4, n_merged.columns[[1] + list(range(5, n_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,4.0,Dog Run,Mediterranean Restaurant,Pool,Golf Course,Fried Chicken Joint,Dim Sum Restaurant,Comfort Food Restaurant,Construction & Landscaping,Furniture / Home Store,Convenience Store
1,North York,4.0,Clothing Store,Fast Food Restaurant,Coffee Shop,Restaurant,Kids Store,Toy / Game Store,Tea Room,Metro Station,Bakery,Japanese Restaurant
5,North York,4.0,Ramen Restaurant,Coffee Shop,Sushi Restaurant,Restaurant,Sandwich Place,Japanese Restaurant,Café,Pet Store,Bubble Tea Shop,Plaza
7,North York,4.0,Grocery Store,Pizza Place,Discount Store,Coffee Shop,Pharmacy,Asian Restaurant,Electronics Store,Convenience Store,Cosmetics Shop,Cupcake Shop
10,North York,4.0,Gym,Asian Restaurant,Coffee Shop,Beer Store,Grocery Store,Smoke Shop,Fast Food Restaurant,Italian Restaurant,Japanese Restaurant,Discount Store
11,North York,4.0,Coffee Shop,Supermarket,Middle Eastern Restaurant,Frozen Yogurt Shop,Pharmacy,Pizza Place,Deli / Bodega,Restaurant,Bridal Shop,Sandwich Place
12,North York,4.0,Coffee Shop,Miscellaneous Shop,Caribbean Restaurant,Metro Station,Massage Studio,Bar,Falafel Restaurant,Discount Store,Cosmetics Shop,Cupcake Shop
14,North York,4.0,Grocery Store,Bank,Shopping Mall,Electronics Store,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store
17,North York,4.0,Hockey Arena,Portuguese Restaurant,Intersection,Coffee Shop,Women's Store,Dog Run,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop
18,North York,4.0,Coffee Shop,Fast Food Restaurant,Italian Restaurant,Grocery Store,Indian Restaurant,Liquor Store,Cupcake Shop,Café,Pharmacy,Pizza Place
